## Library

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import albumentations as A
from torch.utils.data import DataLoader, Dataset
import os
import cv2
from albumentations.pytorch import ToTensorV2
import torch.nn.functional as F
import math
from torch.nn import Parameter
import torch.optim as optim
import torchvision.transforms as transforms
import time
import torch.nn as nn
from torch.nn import Linear, Conv2d, BatchNorm1d, BatchNorm2d, ReLU, Dropout, MaxPool2d, Sequential, Module

## Check data

In [ ]:
# path data

path_data = 'data/train'

In [ ]:
folder = os.listdir(path_data)
num_classes = len(folder)
print("Number of people:", num_classes)

## Set up transform

In [ ]:
INPUT_SIZE = (112,112)

# refer to https://pytorch.org/docs/stable/torchvision/transforms.html for more build-in online data augmentation

data_transform = transforms.Compose([ 
        transforms.ToTensor(),
        transforms.Resize([int(128 * INPUT_SIZE[0] / 112), int(128 * INPUT_SIZE[0] / 112)]), # smaller side resized
        transforms.RandomCrop([INPUT_SIZE[0], INPUT_SIZE[1]]),
        transforms.RandomHorizontalFlip(),
        transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),

    ])

## Data prepare

In [ ]:
images = []
labels = []

In [ ]:
folder.sort()
print(len(folder))

for i in range(len(folder)):
    path_singer = os.path.join(path_data, folder[i])
    for j in os.listdir(path_singer):
        path_img = os.path.join(path_singer, j)
        img = cv2.imread(path_img)
        img = cv2.resize(img, INPUT_SIZE)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        images.append(img)
        labels.append(i)

In [ ]:
print("Number of images:", len(images))
print("Number of labels:", len(labels))
print(f"Shape of images array: {np.array(images).shape}")
print(f"Shape of labels array: {np.array(labels).shape}")

## Data Shuffle

I have prepared a separate train and test set so there is no need to split it into train set and test set.

In [ ]:
from sklearn.utils import shuffle

# Shuffle the training data
X_train, y_train = shuffle(images, labels, random_state=42)

print("Number of train image:", len(X_train))
print("Number of train label:", len(y_train))

In [ ]:
# Show image
plt.imshow(X_train[0])
plt.show()

# Data Read

In [ ]:
class DataBasic(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]
        label = self.labels[idx]

        if self.transform is not None:
            img = self.transform(img)

        return img, label

In [ ]:
data_train = DataBasic(X_train, y_train, data_transform)

## Dataloader

In [ ]:
num_workers = os.cpu_count()

train_loader = DataLoader(dataset = data_train,
                         batch_size= 16,
                         shuffle = False,
                         num_workers = num_workers,
                         )

In [ ]:
# Show image
x = next(iter(train_loader))
img = x[0][2]
plt.imshow(img.permute(1, 2, 0))
plt.show()

## Backbone

In [4]:
INPUT_SIZE = (112,112)

In [5]:
from backbone.model_irse import IR_SE_50
BACKBONE = IR_SE_50(INPUT_SIZE)

## Loss

In [ ]:
EMBEDDING_SIZE = 512

NUM_CLASS = len(np.unique(labels))
print(f'NUM_CLASS : {NUM_CLASS}')

GPU_ID = [0]

# STAGES = [35, 65, 95]
# LR = 0.1  # initial LR
# WEIGHT_DECAY = 5e-4  # do not apply to batch_norm parameters
# MOMENTUM = 0.9
# BATCH_SIZE = 16
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [10]:
from loss.loss_ArcFace import ArcFace
from loss.loss_Focal import FocalLoss
from utils import separate_resnet_bn_paras, schedule_lr

HEAD = ArcFace(in_features=EMBEDDING_SIZE, out_features=NUM_CLASS, device_id=GPU_ID)
LOSS = FocalLoss()

# Separate parameters
backbone_paras_only_bn, backbone_paras_wo_bn = separate_resnet_bn_paras(BACKBONE)
_, head_paras_wo_bn = separate_resnet_bn_paras(HEAD)


In [ ]:
from torch.optim.lr_scheduler import StepLR

# Initialize optimizer
LR = 0.0001

OPTIMIZER = optim.Adam([{'params': backbone_paras_wo_bn + head_paras_wo_bn}, {'params': backbone_paras_only_bn}], lr=LR)

# OPTIMIZER = optim.SGD([{'params': backbone_paras_wo_bn + head_paras_wo_bn, 'weight_decay': WEIGHT_DECAY}, {'params': backbone_paras_only_bn}], lr = LR, momentum = MOMENTUM)

# # Initialize scheduler
scheduler = StepLR(OPTIMIZER, step_size=65, gamma=0.1)

## Move models to device

In [ ]:
HEAD = HEAD.to(DEVICE)
BACKBONE = BACKBONE.to(DEVICE)

## Training - Save check point

In [ ]:
# Hyperparameters
num_epochs = 300
total_step = len(train_loader)
print(f'total_step: {total_step}')

In [ ]:
# Training
import time

BACKBONE.train()  # Set to training mode
HEAD.train()

for e in range(num_epochs):
    print('Epoch:', e + 1)
    LOSSES = []
    start = time.time()

    for idx, (images, labels) in enumerate(train_loader):
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)

        # Forward pass
        features = BACKBONE(images)
        outputs = HEAD(features, labels)

        loss = LOSS(outputs, labels)

        # Backward pass and optimization
        OPTIMIZER.zero_grad()
        loss.backward()
        OPTIMIZER.step()

        LOSSES.append(loss.item())

    # if e == STAGES[0]: # adjust LR for each training stage after warm up, you can also choose to adjust LR manually (with slight modification) once plaueau observed
    #     schedule_lr(OPTIMIZER)
    # if e == STAGES[1]:
    #     schedule_lr(OPTIMIZER)
    # if e == STAGES[2]:
    #     schedule_lr(OPTIMIZER)

    # Adam
    scheduler.step()

    loss_train = np.mean(LOSSES)
    end = time.time()

    # Save the model checkpoint
    if (e + 1) % 10 == 0:
        BACKBONE_path = f'checkpoint/ISE_{e + 1}.pth'
        torch.save(BACKBONE, BACKBONE_path)
        print(f"Saved {BACKBONE_path}")

    print(f'Train loss: {loss_train:.4f} Time: {end - start:.2f} s')

print("-"*50)
print("End")